<p align = "justify">
Here, we will write the same constraint class except we will now use Gurobi instead of Python OR tools. The benefit of the Gurobi is that it runs significantly faster than the Python OR tools.
</p>

In [ ]:
!pip install gurobipy

     |████████████████████████████████| 11.5 MB 13.8 MB/s 


In [ ]:
from gurobipy import *
import random

class MLBSchedulingGurobi(object):
  def __init__(self):
    ## based on the MLB scheduling Wikipedia, we have 52 series (2 series per week)
    self.model = Model()
    self.assignments = {}
    self.series = range(1,53)
    self.central_division = {'WhiteSox':'Chicago','Guardians':'Cleveland','Tigers':'Detroit','Royals':'Kansas City',
                                  'Twins':'Minnosota','Clubs':'Chicago','Reds':'Cincinnati','Brewers':'Milwaukee',
                                  'Pirates':'Pittsburgh','Cardinals':'St.Louis'}

    self.west_division = {'Astros':'Houston','Angels':'Los Angeles','Athletics':'Oakland','Mariners':'Seattle',
                      'Rangers':'Texas','Diamondbacks':'Arizona','Rockies':'Colorado','Dodgers':'Los Angeles',
                      'Padres':'San Diego','Giants':'San Francisco'}

    self.east_division = {'Orioles':'Baltimore','Braves':'Atlanta','RedSox':'Boston','Marlins':'Miami','Yankees':'New York',
                      'Mets':'New York','Rays':'Tampa Bay','Phillies':'Philadelphia','BlueJays':'Toronto','Nationals':'Washington'}

    self.all_divisions = [self.central_division,self.west_division,self.east_division]

    # all MLB teams
    self.teams = list(self.central_division.keys()) + list(self.east_division.keys()) + list(self.west_division.keys())
    
    #keeping track of constraints added, so that we can delete or modify 

    for s in self.series:
      for away_team in self.teams:
        for home_team in self.teams:
          if away_team != home_team:
            value = self.model.addVar(vtype = GRB.BINARY, name = 'X' + str(s) + '_' + away_team + '_' + home_team)
            self.assignments[(s,away_team,home_team)] = value

  def test(self): 
    print("tested successfully")
  ## Each team plays a certain number of inter-divisional games
  def numInterDivisionalGames(self,inter_divisional_games):
    for division in self.all_divisions:
      for team in division.keys():
        sum = 0
        for other_team in self.teams:
          if other_team not in division.keys():
            for s in self.series:
              if s % 2 == 0:
                sum = sum + 4 * (self.assignments[(s,team,other_team)] + self.assignments[(s,other_team,team)])
              else:
                sum = sum + 3 * (self.assignments[(s,team,other_team)] + self.assignments[(s,other_team,team)])
        self.model.addConstr(sum >= inter_divisional_games)

  ## Each teach plays a number of intra-divisional games
  def numIntraDivisionalGames(self,intra_divisional_games):
    for division in self.all_divisions:
      for team in division.keys():
        sum = 0
        for other_team in self.teams:
          if other_team in division.keys() and team != other_team:
            for s in self.series:
              if s % 2 == 0:
                sum = sum + 4 * (self.assignments[(s,team,other_team)] + self.assignments[(s,other_team,team)])
              else:
                sum = sum + 3 * (self.assignments[(s,team,other_team)] + self.assignments[(s,other_team,team)])
        self.model.addConstr(sum >= intra_divisional_games)
  
  ## Each team plays in each slot (series).
  def playInEverySlot(self):
    for s in self.series:
      for curr_team in self.teams:
        sum = 0
        for opposing_team in self.teams:
          if curr_team != opposing_team:
            sum = sum + self.assignments[(s,curr_team,opposing_team)] + self.assignments[(s,opposing_team,curr_team)] 
        self.model.addConstr(sum == 1)

  ## Each slot (or game) is home or away (interpreation: if one team plays another during one slot (or game)
  # , one is home and another is away)
  def setHomeAwayPerGame(self):
    for s in self.series:
      for i in range(len(self.teams)):
        for j in range(i+1,len(self.teams)):
          curr_team = self.teams[i]
          opposing_team = self.teams[j]
          sum = self.assignments[(s,curr_team,opposing_team)] + self.assignments[(s,opposing_team,curr_team)]
          self.model.addConstr(sum <= 1)
  
  ## Modify the previous constraint on the exact number of home games to at least a certain
  ## number of home games
  ## Based on the Wikipedia for the MLB scheduling, we have series consisting of 3 and 4 games
  ## For this example, we will define two series per week: 1st one Monday - Wednesday
  ## Also, we will define the second series: 2nd one Thursday - Sunday
  ## Each series we either have 3 or 4 games (namely for odd series, we will use 3 games
  ## and for even number series, we will use 4 games)
  def setNumHomeGamesMinimum(self,min_home_games):
    for home_team in self.teams:
      sum = 0
      for away_team in self.teams:
        if home_team != away_team:
          for s in self.series:
            if s % 2 == 0: ## even number series
              sum = sum + 4 * self.assignments[(s,away_team,home_team)]
            else:
              sum = sum + 3 * self.assignments[(s,away_team,home_team)]
      self.model.addConstr(sum >= min_home_games)

  ## Here we will set the number of Friday and Saturday home games as another constraint
  ## each of the even numbered series has 2 Friday and Saturday games
  def setFridayAndSaturdayHomeGames(self,num_games):
    for home_team in self.teams:
      sum = 0
      for away_team in self.teams:
        if home_team != away_team:
          for s in self.series:
            if s % 2 == 0: ## even number series
              sum = sum + 2 * self.assignments[(s,away_team,home_team)]
      setFriSatHomeGames = self.model.addConstr(sum >= num_games)
      print(setFriSatHomeGames)

  ## No back to back series between each team
  def noBacktoBackSeries(self):
    for team in self.teams:
      for opposing_team in self.teams:
        if team != opposing_team:
          for j in range(1,52):
            curr_series = j
            next_series = j + 1

            ## Note if team RedSox plays Astros on Series 1
            ## RedSox cannot play Astros nor can Astros play RedSox on Series 2
            self.model.addConstr(self.assignments[(curr_series,team,opposing_team)] 
                            + self.assignments[(next_series,team,opposing_team)] <= 1)
            self.model.addConstr(self.assignments[(curr_series,team,opposing_team)] 
                            + self.assignments[(next_series,opposing_team,team)] <= 1)



  #deleting constraints for the model
  def deleteConstraint(self): 
    #choose by random which constraint to delete
    constraints = self.model.getConstrs()[0:]

    #adjust it so that it accepts user input 
    del_constraint = random.choice(constraints)
    
    #delete  
    self.model.remove(del_constraint) #from beginning to end 


  ## Here we define the objective function for the problem
  def setObjective(self):
      ## Defining the objective function here
      objective = 0.0
      for s in self.series:
        for away_team in self.teams:
          for home_team in self.teams:
            if home_team != away_team:
              objective = objective + self.assignments[(s,away_team,home_team)]
      return objective

  def solve(self,objective):
      ## solver solves the binary integer programming problem
      self.model.setObjective(objective,GRB.MAXIMIZE)
      self.model.optimize()
      '''status = self.solver.Solve()

      if status == pywraplp.Solver.OPTIMAL: # Optimal solution
        for key,value in self.assignments.items():
          # only print the decision variables that equal 1.0
          if value.solution_value() == 1.0:
            print(key,value.solution_value())
        print('The optimal objective function value is ',self.solver.Objective().Value())

      elif status == pywraplp.Solver.INFEASIBLE: #Infeasible solution
        print('the problem is infeasible')
      else:
        print('the solution is not optimal')'''

In [ ]:
gurobi_test = MLBSchedulingGurobi()
gurobi_test.numInterDivisionalGames(40)
gurobi_test.setHomeAwayPerGame()
gurobi_test.deleteConstraint()

IndexError: ignored